In [1]:
import os
import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import  Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def make_model():
    k = float(np.random.rand()*1+0.2)
    print ('## k = %.3f' % k)
    winit1 = k/np.sqrt(5*5*1)
    winit2 = k/np.sqrt(5*5*64)
    winit3 = k/np.sqrt(5*5*128)


    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=5, padding='same',
                    data_format='channels_first',
                    use_bias=False, 
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=winit1),
                    activation='relu', input_shape=(1,96,96)))
    model.add(MaxPooling2D(pool_size=2, strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=5, padding='same',
                    data_format='channels_first',
                    use_bias=False,
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=winit2),
                    activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=5, padding='same',
                    data_format='channels_first',
                    use_bias=False,
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=winit3),
                    activation='relu'))
    model.add(MaxPooling2D(pool_size=12, strides=(12,12)))
    model.add(Flatten())

    winitD1 = k/np.sqrt(4032)
    winitD2 = k/np.sqrt(300)
    model.add(Dropout(0.5))
    model.add(Dense(300, activation='relu',
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05),
                    bias_initializer='Ones'))

    model.add(Dense(8, activation='softmax',
    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=winitD2),
    bias_initializer='Zeros'))

    print(model.summary())
    
    # compile the model
    sgd = keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=sgd,
             metrics=['accuracy'])

    return model

In [3]:
def make_cv_folds(X, y, folds, fold_num):
    X_train = []
    y_train = []
    X_valid = []
    y_valid = []

    for index, fold in enumerate(folds):
        if fold == fold_num:
            X_valid.append(X[index])
            y_valid.append(y[index])
        else:
            X_train.append(X[index])
            y_train.append(y[index])
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_valid = np.array(X_valid)
    y_valid = np.array(y_valid)


    return X_train, y_train, X_valid, y_valid

In [4]:
# Data loading
dataset_path = 'save_data/npy_files'
result_path = 'result/'
X = np.load(os.path.join(dataset_path,'X.npy'))
y = np.load(os.path.join(dataset_path,'y.npy'))
folds = np.load(os.path.join(dataset_path,'folds.npy'))

# rescale [0,255] -> [0,2]    
X = X.astype('float32')/255*2

# one-hot encode the labels
num_classes = len(np.unique(y))
y = keras.utils.to_categorical(y, num_classes)

In [5]:
result_path = './result/test'
val_fold = 0
X_train, y_train, X_valid, y_valid = make_cv_folds(X, y, folds, val_fold)

print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)


# printing number of training, validation, and test images
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'validation samples')
#X_test = X_test.astype('float32')/255
model = make_model()
filepath="weights-{epoch:02d}-{val_acc:.2f}.hdf5"
filepath = os.path.join(result_path, filepath)
checkpointer = ModelCheckpoint(filepath=filepath,
                        verbose=1, save_best_only=True)

hist = model.fit(X_train, y_train, batch_size=64, epochs=1000,
            validation_data=(X_valid, y_valid),
            callbacks=[checkpointer], verbose=2, shuffle=True)

X_train shape : (1175, 1, 96, 96)
y_train shape : (1175, 8)
1175 train samples
132 validation samples
## k = 0.407
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 96, 96)        1600      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 48, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 48, 96)       102400    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 24, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 24, 96)       409600    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 21, 2, 96)         0         
___________________________

Epoch 00044: val_loss did not improve
 - 1s - loss: 0.1575 - acc: 0.9506 - val_loss: 0.7224 - val_acc: 0.8106
Epoch 45/1000
Epoch 00045: val_loss did not improve
 - 1s - loss: 0.0905 - acc: 0.9711 - val_loss: 1.1513 - val_acc: 0.7576
Epoch 46/1000
Epoch 00046: val_loss did not improve
 - 1s - loss: 0.0982 - acc: 0.9668 - val_loss: 0.9833 - val_acc: 0.7879
Epoch 47/1000
Epoch 00047: val_loss did not improve
 - 1s - loss: 0.0715 - acc: 0.9728 - val_loss: 0.8027 - val_acc: 0.7803
Epoch 48/1000
Epoch 00048: val_loss did not improve
 - 1s - loss: 0.0625 - acc: 0.9770 - val_loss: 0.7706 - val_acc: 0.8182
Epoch 49/1000
Epoch 00049: val_loss did not improve
 - 1s - loss: 0.0582 - acc: 0.9804 - val_loss: 0.9588 - val_acc: 0.7803
Epoch 50/1000
Epoch 00050: val_loss did not improve
 - 1s - loss: 0.0780 - acc: 0.9728 - val_loss: 0.7089 - val_acc: 0.8106
Epoch 51/1000
Epoch 00051: val_loss did not improve
 - 1s - loss: 0.1376 - acc: 0.9455 - val_loss: 0.6225 - val_acc: 0.8485
Epoch 52/1000
Epoch 00

Epoch 00110: val_loss did not improve
 - 1s - loss: 0.0583 - acc: 0.9804 - val_loss: 1.3382 - val_acc: 0.8030
Epoch 111/1000
Epoch 00111: val_loss did not improve
 - 1s - loss: 0.0476 - acc: 0.9872 - val_loss: 1.0950 - val_acc: 0.8409
Epoch 112/1000
Epoch 00112: val_loss did not improve
 - 1s - loss: 0.0463 - acc: 0.9864 - val_loss: 1.3049 - val_acc: 0.7727
Epoch 113/1000
Epoch 00113: val_loss did not improve
 - 1s - loss: 0.0430 - acc: 0.9855 - val_loss: 0.9130 - val_acc: 0.8712
Epoch 114/1000
Epoch 00114: val_loss did not improve
 - 1s - loss: 0.0270 - acc: 0.9898 - val_loss: 0.9718 - val_acc: 0.8485
Epoch 115/1000
Epoch 00115: val_loss did not improve
 - 1s - loss: 0.0070 - acc: 0.9974 - val_loss: 0.9674 - val_acc: 0.8333
Epoch 116/1000
Epoch 00116: val_loss did not improve
 - 1s - loss: 0.0065 - acc: 0.9966 - val_loss: 1.0521 - val_acc: 0.8258
Epoch 117/1000
Epoch 00117: val_loss did not improve
 - 1s - loss: 0.0274 - acc: 0.9932 - val_loss: 0.9715 - val_acc: 0.8712
Epoch 118/1000


Epoch 176/1000
Epoch 00176: val_loss did not improve
 - 1s - loss: 0.0182 - acc: 0.9949 - val_loss: 0.9863 - val_acc: 0.8485
Epoch 177/1000
Epoch 00177: val_loss did not improve
 - 1s - loss: 0.0194 - acc: 0.9949 - val_loss: 1.6206 - val_acc: 0.8258
Epoch 178/1000
Epoch 00178: val_loss did not improve
 - 1s - loss: 0.0158 - acc: 0.9957 - val_loss: 1.6848 - val_acc: 0.8258
Epoch 179/1000
Epoch 00179: val_loss did not improve
 - 1s - loss: 0.0305 - acc: 0.9940 - val_loss: 1.6052 - val_acc: 0.7879
Epoch 180/1000
Epoch 00180: val_loss did not improve
 - 1s - loss: 0.0357 - acc: 0.9906 - val_loss: 1.6805 - val_acc: 0.8485
Epoch 181/1000
Epoch 00181: val_loss did not improve
 - 1s - loss: 0.0179 - acc: 0.9974 - val_loss: 1.2048 - val_acc: 0.8182
Epoch 182/1000
Epoch 00182: val_loss did not improve
 - 1s - loss: 0.0096 - acc: 0.9974 - val_loss: 1.2907 - val_acc: 0.7803
Epoch 183/1000
Epoch 00183: val_loss did not improve
 - 1s - loss: 0.0194 - acc: 0.9915 - val_loss: 1.7141 - val_acc: 0.7652


Epoch 242/1000
Epoch 00242: val_loss did not improve
 - 1s - loss: 0.0194 - acc: 0.9940 - val_loss: 1.4883 - val_acc: 0.8030
Epoch 243/1000
Epoch 00243: val_loss did not improve
 - 1s - loss: 0.0063 - acc: 0.9983 - val_loss: 0.7274 - val_acc: 0.7803
Epoch 244/1000
Epoch 00244: val_loss did not improve
 - 1s - loss: 0.0060 - acc: 0.9983 - val_loss: 1.7272 - val_acc: 0.7879
Epoch 245/1000
Epoch 00245: val_loss did not improve
 - 1s - loss: 0.0032 - acc: 0.9983 - val_loss: 1.4136 - val_acc: 0.8106
Epoch 246/1000
Epoch 00246: val_loss did not improve
 - 1s - loss: 0.0036 - acc: 0.9991 - val_loss: 1.1810 - val_acc: 0.8182
Epoch 247/1000
Epoch 00247: val_loss did not improve
 - 1s - loss: 0.0098 - acc: 0.9974 - val_loss: 1.6739 - val_acc: 0.7803
Epoch 248/1000
Epoch 00248: val_loss did not improve
 - 1s - loss: 0.0252 - acc: 0.9940 - val_loss: 1.5544 - val_acc: 0.8561
Epoch 249/1000
Epoch 00249: val_loss did not improve
 - 1s - loss: 0.0126 - acc: 0.9966 - val_loss: 1.3461 - val_acc: 0.7955


...

Epoch 699/1000
Epoch 00699: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 700/1000
Epoch 00700: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 701/1000
Epoch 00701: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 702/1000
Epoch 00702: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 703/1000
Epoch 00703: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 704/1000
Epoch 00704: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 705/1000
Epoch 00705: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 706/1000
Epoch 00706: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - 

Epoch 764/1000
Epoch 00764: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 765/1000
Epoch 00765: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 766/1000
Epoch 00766: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 767/1000
Epoch 00767: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 768/1000
Epoch 00768: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 769/1000
Epoch 00769: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 770/1000
Epoch 00770: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 771/1000
Epoch 00771: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - 

Epoch 829/1000
Epoch 00829: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 830/1000
Epoch 00830: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 831/1000
Epoch 00831: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 832/1000
Epoch 00832: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 833/1000
Epoch 00833: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 834/1000
Epoch 00834: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 835/1000
Epoch 00835: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 836/1000
Epoch 00836: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - 

Epoch 894/1000
Epoch 00894: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 895/1000
Epoch 00895: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 896/1000
Epoch 00896: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 897/1000
Epoch 00897: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 898/1000
Epoch 00898: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 899/1000
Epoch 00899: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 900/1000
Epoch 00900: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 901/1000
Epoch 00901: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - 

Epoch 959/1000
Epoch 00959: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 960/1000
Epoch 00960: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 961/1000
Epoch 00961: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 962/1000
Epoch 00962: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 963/1000
Epoch 00963: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 964/1000
Epoch 00964: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 965/1000
Epoch 00965: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 966/1000
Epoch 00966: val_loss did not improve
 - 1s - loss: 12.0577 - acc: 0.2519 - val_loss: 12.0886 - 

In [ ]:
hist.history['acc'][41]

In [ ]:
hist.history['val_acc'][41]

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
with open('./result/test/model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
from keras.models import model_from_json

In [ ]:
with open('./result/test/model_architecture.json', 'r') as f:
    model2 = model_from_json(f.read())

model2.load_weights('./result/test/weights-42-0.90.hdf5')

In [ ]:
model2.summary()